# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
# Part Three

Dataframe *neighbourhoods* was built in the second part of this assignment. 

*neighbourhoods* contains the postal code of each neighborhood along with the borough name, neighborhood name, lattitude and longittude coordinates.

Hidden cell below contains code for importing dataset (with client credentials).

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0




### Importing libraries

In [86]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Importing dataframe created in Part 1

In [87]:
neighbourhoods = pd.read_csv(body)
neighbourhoods.head(10)


,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
1,1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
2,2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
3,3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,4,M5N,Central Toronto,Roselawn,43.711695,-79.416936
5,5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,6,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
7,7,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
8,8,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
9,9,M6G,Downtown Toronto,Christie,43.669542,-79.422564


In [88]:
#Removing "Unnamed: 0" index column
neighbourhoods.drop(['Unnamed: 0'], axis=1)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M5N,Central Toronto,Roselawn,43.711695,-79.416936
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
7,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
8,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
9,M6G,Downtown Toronto,Christie,43.669542,-79.422564


### Creating Folium map

In [89]:
# create map of Toronto using latitude and longitude values for Toronto City Hall
latitude =  43.653908
longitude = -79.384293
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

## Toronto Exploration

### Defining Foursqaure Credentials

In [90]:
CLIENT_ID = 'PGNGQ1TOCDTFFEFA4FEPJY5RBOKTSFI13LLGLOL0C4BXMH2O' # your Foursquare ID
CLIENT_SECRET = 'RZZ1F2N1RBGCP0SEFHSZCRSHIVFBB3NDSDJB3LRFVHY53LZN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PGNGQ1TOCDTFFEFA4FEPJY5RBOKTSFI13LLGLOL0C4BXMH2O
CLIENT_SECRET:RZZ1F2N1RBGCP0SEFHSZCRSHIVFBB3NDSDJB3LRFVHY53LZN


### Defining function generating list of venues through API request

The function below produces 50 venues for each neighbourhood in 500 meters radius.

In [91]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



Function call with *neighbourhoods* dataframe will provide information on venues near each neighbourhood (name, coordinates and category).

In [92]:
toronto_venues = getNearbyVenues(
                    names = neighbourhoods['Neighbourhood'],
                    latitudes = neighbourhoods['Latitude'],
                    longitudes = neighbourhoods['Longitude']
                                )
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Moore Park, Summerhill East",43.689574,-79.38316,Loring-Wyle Parkette,43.690270,-79.383438,Park
1,"Moore Park, Summerhill East",43.689574,-79.38316,Moore Park Tennis Club,43.693289,-79.382900,Tennis Court
2,Lawrence Park,43.728020,-79.38879,Lawrence Park Ravine,43.726963,-79.394382,Park
3,Lawrence Park,43.728020,-79.38879,Zodiac Swim School,43.728532,-79.382860,Swim School
4,Lawrence Park,43.728020,-79.38879,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line


In [93]:
toronto_venues.shape

(1700, 7)

Total number of venues generated is 1071.

### Venue statistics

The most popular neighbourhoods in Toronto based on number of venues

In [94]:
df1 = toronto_venues.groupby(['Neighbourhood']).count().reset_index()

df1.rename(columns={'Venue': 'Number of venues'}, inplace=True)

df1 = df1[['Neighbourhood', 'Number of venues']].sort_values(by=['Number of venues'], ascending = False).reset_index(drop=True).head(20)
df1


,Neighbourhood,Number of venues
0,"Adelaide, King, Richmond",50
1,"Little Portugal, Trinity",50
2,"Chinatown, Grange Park, Kensington Market",50
3,Central Bay Street,50
4,"Design Exchange, Toronto Dominion Centre",50
5,"Fairview, Henry Farm, Oriole",50
6,"First Canadian Place, Underground city",50
7,"Harbourfront East, Toronto Islands, Union Station",50
8,Church and Wellesley,50
9,"Commerce Court, Victoria Hotel",50


Note: Top neighbourhoods might have even higher numbers of venues but it was limitted to 50 during request function.

Next step is to check what types of venues are most popular

In [95]:
df2 = toronto_venues.groupby(['Venue Category']).count().reset_index()
df2.rename(columns={'Venue': 'Number of venues'}, inplace=True)
df2[['Venue Category', 'Number of venues']].sort_values(by=['Number of venues'], ascending = False).reset_index(drop=True).head(10)




,Venue Category,Number of venues
0,Coffee Shop,135
1,Café,85
2,Park,50
3,Pizza Place,47
4,Restaurant,45
5,Bakery,37
6,Sandwich Place,34
7,Italian Restaurant,32
8,Bar,28
9,Gym,27


Another useful information could be what venues are the most rare

In [96]:
df2 = toronto_venues.groupby(['Venue Category']).count().reset_index()
df2.rename(columns={'Venue': 'Number of venues'}, inplace=True)
df2[['Venue Category', 'Number of venues']].sort_values(by=['Number of venues']).reset_index(drop=True).head(20)





,Venue Category,Number of venues
0,Accessories Store,1
1,Indoor Play Area,1
2,Indonesian Restaurant,1
3,Indie Movie Theater,1
4,Hotel Bar,1
5,Hostel,1
6,Historic Site,1
7,Health & Beauty Service,1
8,Hardware Store,1
9,Harbor / Marina,1


Since there are many categories, some venues types displayed above could also be categorized by the most popular categories (like Restaurant for Indonesian Restaurant or Hakka Restaurant, Gaming Cafe for Cafe etc.) but this kind of data cleaning is out of this projects' scope. 

### Analyzing neighbourhoods

Performing One Hot Encoding for different types of venues for each neighbourhood

In [97]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [98]:
toronto_onehot.shape

(1700, 258)

Grouping rows by neighborhood with the mean of the frequency of occurrence of each category

In [99]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0,0.02,0.02,0.0,0.0,0.0,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.080000,0.0,0.0,0.0,0.0,0.0,0.

Calculating frequency of venue type in each neighbourhood and 3 top venues per neighbourhood

In [100]:
num_top_venues = 3
for neigh in toronto_grouped['Neighbourhood']:
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted = neighborhoods_venues_sorted[neighborhoods_venues_sorted['1st Most Common Venue'] != 'NaN']

neighborhoods_venues_sorted.head()




,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Café
1,Agincourt,Lounge,Skating Rink,Breakfast Spot
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pool


In [101]:
neighborhoods_venues_sorted.shape


(99, 4)

## Clustering neighbourhoods

### Model preparation

In [102]:
# set number of clusters
k = 10

toronto_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 8, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Integrating created labels with *neighbourhoods* dataframe

In [103]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_neighbourhoods = neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_neighbourhoods = toronto_neighbourhoods.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='right')

toronto_neighbourhoods.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,8,Tennis Court,Park,Drugstore
1,1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Bus Line,Park,Swim School
2,2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Coffee Shop,Pub,Bagel Shop
3,3,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Coffee Shop
4,4,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Home Service,Garden,Pool


### Creating map with clusters to visualize performed clustering

In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.RdBu(np.linspace(0, 1, len(ys)))
RdBu = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_neighbourhoods['Latitude'], toronto_neighbourhoods['Longitude'], toronto_neighbourhoods['Neighbourhood'], toronto_neighbourhoods['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=RdBu[cluster-1],
        fill=True,
        fill_color=RdBu[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters